To use wyscout with the package SoccerAction you have to set wyscout as your data provider with this cell

In [ ]:
from socceraction.data.wyscout import WyscoutLoader

# set authentication credentials as environment variables
import os
os.environ["WY_USERNAME"] = "your_username"
os.environ["WY_PASSWORD"] = "your_password"
api = WyscoutLoader(getter="remote")

# or provide authentication credentials as a dictionary
#api = WyscoutLoader(getter="remote", creds={"user": "", "passwd": ""})

And this is for set the lenght of the field when you use spadl with SoccerAction

In [ ]:
import socceraction.spadl as spadl
spadl.config.field_length : float = 100.0
spadl.config.field_width : float = 100.0

# SoccerAction Functions

In [ ]:
def get_ball_recovery_socceraction(team_wyid, competition_wyid, season_wyid = ''):
  shot = []
  shots_df = pd.DataFrame(shot)
  games = tq(api.games(competition_wyid, season_wyid))
  for i, m in tq(games.iterrows(), desc = 'Progresso partite processate recupero palla', total = len(games)):
    if m['home_team_id'] == team_wyid or m['away_team_id'] == team_wyid:
      events = api.events(m['game_id'])
      if m['home_team_id'] == team_wyid:
        home_team_id = team_wyid
      else:
        home_team_id = m['home_team_id']
      actions = spadl.wyscout.convert_to_actions(events, home_team_id)
      actions = spadl.add_names(actions)
      for j, e in actions.iterrows():
        if (e['type_name'] == 'shot' and e ['team_id'] == team_wyid):
          last_opponent = ((actions.loc[j::-1, 'team_id'] != team_wyid)).idxmax()
          if j-last_opponent <= 7:
            z = True
            for x in range(j, last_opponent, -1):
              if actions.loc[x, 'type_name'] == 'foul' or actions.loc[x, 'team_id'] != team_wyid:
                z = False
                break
          else:
            next
        if z:
          if actions.loc[last_opponent, 'end_x'] < 66:
            if actions.loc[j, 'start_x'] - actions.loc[last_opponent, 'end_x'] >25:
              if actions.loc[last_opponent, 'end_x'] < 38:
                if actions.loc[j, 'time_seconds'] - actions.loc[last_opponent, 'time_seconds'] <= 15:
                  e['shotType'] = 'CounterAttackShot'
                  shot.append(e)
                elif actions.loc[last_opponent, 'end_x'] > 38 and actions.loc[last_opponent, 'end_x'] < 66:
                  if actions.loc[i, 'time_seconds'] - actions.loc[last_opponent, 'time_seconds'] <= 12:
                    e['shotType'] = 'CounterAttackShot'
                    shot.append(e)
          else:
            if j-last_opponent <= 4 and actions.loc[last_opponent, 'type_name'] == 'interception':
              if 100 - actions.loc[last_opponent, 'end_x'] <=25:
                e['shotType'] = 'BallRecoveryShot'
                shot.append(e)







  shots_df = pd.DataFrame(shot)
  return shots_df

In [ ]:
def get_corner_kick_schema_socceraction(team_wyid, competition_wyid, season_wyid = ''):
  shot = []
  shots_df = pd.DataFrame(shot)
  games = api.games(competition_wyid, season_wyid)
  for i, m in tq(games.iterrows(), desc = 'Progresso partite processate calcio d\'angolo indiretto', total = len(games)):
    if m['home_team_id'] == team_wyid or m['away_team_id'] == team_wyid:
      events = api.events(m['game_id'])
      if m['home_team_id'] == team_wyid:
        home_team_id = team_wyid
      else:
        home_team_id = m['home_team_id']
      actions = spadl.wyscout.convert_to_actions(events, home_team_id)
      actions = spadl.add_names(actions)
      for j, e in actions.iterrows():
        if (e['type_name'] == 'shot' and e ['team_id'] == team_wyid):
          first_corner = ((actions.loc[j::-1, 'type_name'].eq('corner_crossed')) & (actions.loc[j::-1, 'team_id'].eq(team_wyid))).idxmax()
          first_short = ((actions.loc[j::-1, 'type_name'].eq('corner_short')) & (actions.loc[j::-1, 'team_id'].eq(team_wyid))).idxmax()
          if j-first_corner < j-first_short:
            if actions.loc[first_corner, 'result_name'] == 'success':
              if actions.loc[first_corner +1, 'type_name'] == 'shot':
                e['shotType'] = 'IndirectCornerKickWithCross'
                shot.append(e)
              elif actions.loc[first_corner +1, 'type_name'] == 'dribble' and actions.loc[first_corner +2, 'type_name'] == 'shot':
                e['shotType'] = 'IndirectCornerKickWithCrossAndBounce'
                shot.append(e)
              elif ((actions.loc[first_corner +1, 'type_name'] == 'dribble') or (actions.loc[first_corner +1, 'type_name'] == 'pass')) and ((actions.loc[first_corner +2, 'type_name'] == 'dribble') or (actions.loc[first_corner +2, 'type_name'] == 'pass')) and actions.loc[first_corner +3, 'type_name'] == 'shot':
                e['shotType'] = 'IndirectCornerKickWithCrossAndBank'
                shot.append(e)
          else:
            if actions.loc[first_short, 'result_name'] == 'success':
              if ((actions.loc[first_short +1, 'type_name'] == 'cross') or (actions.loc[first_short +1, 'type_name'] == 'pass')) and actions.loc[first_short +2, 'type_name'] == 'shot':
                e['shotType'] = 'IndirectCornerKickWithTouch'
                shot.append(e)
              elif ((actions.loc[first_short +1, 'type_name'] == 'cross') or (actions.loc[first_short +1, 'type_name'] == 'pass')) and actions.loc[first_short +3, 'type_name'] == 'shot':
                e['shotType'] = 'IndirectCornerKickWithTwoTouches'
                shot.append(e)
          if j-first_corner > 5 or j-first_short > 5:
            c = 0
            if j-first_corner < j-first_short:
              for x in range(j, first_corner, -1):
                if actions.loc[x, 'team_id'] != team_wyid:
                  break
                else:
                  c += 1
              if c > 4:
                e['shotType'] = 'IndirectCornerKickCrossedWithBounces'
                shot.append(e)
                break
            else:
              for x in range(j, first_corner, -1):
                if actions.loc[x, 'team_id'] != team_wyid:
                  break
                else:
                  c += 1
              if c > 4:
                e['shotType'] = 'IndirectCornerKickShortWithBounces'
                shot.append(e)
                break


  shots_df = pd.DataFrame(shot)
  return shots_df

In [ ]:
def get_free_kick_schema_socceraction(team_wyid, competition_wyid, season_wyid = ''):
  shot = []
  shots_df = pd.DataFrame(shot)
  games = api.games(competition_wyid, season_wyid)
  for i, m in tq(games.iterrows(), desc = 'Progresso partite processate calcio d\'angolo indiretto', total = len(games)):
    if m['home_team_id'] == team_wyid or m['away_team_id'] == team_wyid:
      events = api.events(m['game_id'])
      if m['home_team_id'] == team_wyid:
        home_team_id = team_wyid
      else:
        home_team_id = m['home_team_id']
      actions = spadl.wyscout.convert_to_actions(events, home_team_id)
      actions = spadl.add_names(actions)
      for j, e in actions.iterrows():
        if (e['type_name'] == 'shot' and e ['team_id'] == team_wyid):
          first_free_kick = ((actions.loc[j::-1, 'type_name'].eq('freekick_crossed')) & (actions.loc[j::-1, 'team_id'].eq(team_wyid))).idxmax()
          first_short = ((actions.loc[j::-1, 'type_name'].eq('freekick_short')) & (actions.loc[j::-1, 'team_id'].eq(team_wyid))).idxmax()
          if j-first_free_kick < j-first_short:
            if actions.loc[first_free_kick, 'result_name'] == 'success':
              if actions.loc[first_free_kick +1, 'type_name'] == 'shot':
                e['shotType'] = 'IndirectFreeKickWithCross'
                shot.append(e)
              elif actions.loc[first_free_kick +1, 'type_name'] == 'dribble' and actions.loc[first_free_kick +2, 'type_name'] == 'shot':
                e['shotType'] = 'IndirectFreeKickWithCrossAndBounce'
                shot.append(e)
              elif ((actions.loc[first_free_kick +1, 'type_name'] == 'dribble') or (actions.loc[first_free_kick +1, 'type_name'] == 'pass')) and ((actions.loc[first_free_kick +2, 'type_name'] == 'dribble') or (actions.loc[first_free_kick +2, 'type_name'] == 'pass')) and actions.loc[first_free_kick +3, 'type_name'] == 'shot':
                e['shotType'] = 'IndirectFreeKickWithCrossAndBank'
                shot.append(e)
          else:
            if actions.loc[first_short, 'result_name'] == 'success':
              if ((actions.loc[first_short +1, 'type_name'] == 'cross') or (actions.loc[first_short +1, 'type_name'] == 'pass')) and actions.loc[first_short +2, 'type_name'] == 'shot':
                e['shotType'] = 'IndirectFreeKickWithTouch'
                shot.append(e)
              elif ((actions.loc[first_short +1, 'type_name'] == 'cross') or (actions.loc[first_short +1, 'type_name'] == 'pass')) and actions.loc[first_short +3, 'type_name'] == 'shot':
                e['shotType'] = 'IndirectFreeKickWithTwoTouches'
                shot.append(e)
          if j-first_free_kick > 4 or j-first_short > 4:
            c = 0
            if j-first_free_kick < j-first_short:
              for x in range(j, first_free_kick, -1):
                if actions.loc[x, 'team_id'] != team_wyid:
                  break
                else:
                  c += 1
                if c > 4:
                  e['shotType'] = 'IndirectCornerKickCrossedWithBounces'
                  shot.append(e)
                  break
            else:
              for x in range(j, first_free_kick, -1):
                if actions.loc[x, 'team_id'] != team_wyid:
                  break
                else:
                  c += 1
                if c > 4:
                  e['shotType'] = 'IndirectCornerKickShortWithBounces'
                  shot.append(e)
                  break


  shots_df = pd.DataFrame(shot)
  return shots_df

In [ ]:
def get_throw_in_shot_socceraction(team_wyid, competition_wyid, season_wyid = ''):
  shot = []
  shots_df = pd.DataFrame(shot)
  games = api.games(competition_wyid, season_wyid)
  for i, m in tq(games.iterrows(), desc = 'Progresso partite processate calcio d\'angolo indiretto', total = len(games)):
    if m['home_team_id'] == team_wyid or m['away_team_id'] == team_wyid:
      events = api.events(m['game_id'])
      if m['home_team_id'] == team_wyid:
        home_team_id = team_wyid
      else:
        home_team_id = m['home_team_id']
      actions = spadl.wyscout.convert_to_actions(events, home_team_id)
      actions = spadl.add_names(actions)
      for j, e in actions.iterrows():
        if e['type_name'] == 'shot' and e['team_id'] == team_wyid:
          first_throw_in = (actions.loc[j::-1, 'type_name'].eq('throw_in') & actions.loc[j::-1, 'team_id'].eq(team_wyid)).idxmax()
          if i- first_throw_in == 0:
            next
          elif j - first_throw_in <= 6:
            z = True
            for x in range(j, first_throw_in, -1):
                if actions.loc[x, 'team_id'] != team_wyid:
                  z = False
                  break
            if z:
                e['shotType'] = 'shotAfterThrowIn'
                shot.append(e)




  shots_df = pd.DataFrame(shot)
  return shots_df

In [ ]:
def get_positional_attacks_socceraction(team_wyid, competition_wyid, season_wyid = ''):
  shot = []
  shots_df = pd.DataFrame(shot)
  games = api.games(competition_wyid, season_wyid)
  for i, m in tq(games.iterrows(), desc = 'Progresso partite processate calcio d\'angolo indiretto', total = len(games)):
    if m['home_team_id'] == team_wyid or m['away_team_id'] == team_wyid:
      events = api.events(m['game_id'])
      if m['home_team_id'] == team_wyid:
        home_team_id = team_wyid
      else:
        home_team_id = m['home_team_id']
      actions = spadl.wyscout.convert_to_actions(events, home_team_id)
      actions = spadl.add_names(actions)
      for j, e in actions.iterrows():
        z = 0
        if e['type_name'] == 'shot' and e['team_id'] == team_wyid: #and 'shot_after_free_kick' not in e['secondary'] and 'shot_after_corner' not in e['secondary'] and 'shot_after_throw_in' not in e['secondary']
          last_opponent = (actions.loc[j::-1, 'team_id'] != team_wyid).idxmax()
          for x in range(i, last_opponent, -1):
            if (actions.loc[x, 'team_id'] == team_wyid and \
              (actions.loc[x, 'type_name'] != 'throw_in' and actions.loc[x, 'type_name'] != 'freekick_crossed' and actions.loc[x, 'type_name'] != 'freekick_short' and actions.loc[x, 'type_name'] != 'corner_crossed' and actions.loc[x, 'type_name'] != 'corner_short')):
              j +=1
              if j>6:
                e['shotType'] = 'positionalAttack'
                shot.append(e)
                break
            else:
              j = 0
              break


  shots_df = pd.DataFrame(shot)
  return shots_df

In [ ]:
def get_free_kick_shot_socceraction(team_wyid, competition_wyid, season_wyid = ''):
  shot = []
  shots_df = pd.DataFrame(shot)
  games = api.games(competition_wyid, season_wyid)
  for i, m in tq(games.iterrows(), desc = 'Progresso partite processate calcio d\'angolo indiretto', total = len(games)):
    if m['home_team_id'] == team_wyid or m['away_team_id'] == team_wyid:
      events = api.events(m['game_id'])
      if m['home_team_id'] == team_wyid:
        home_team_id = team_wyid
      else:
        home_team_id = m['home_team_id']
      actions = spadl.wyscout.convert_to_actions(events, home_team_id)
      actions = spadl.add_names(actions)
      for j, e in actions.iterrows():
        if e['type_name'] == 'shot_freekick' and e['team_id'] == team_wyid:
          e['shotType'] = 'DirectFreeKick'
          shot.append(e)


  shots_df = pd.DataFrame(shot)
  return shots_df

In [ ]:
def get_penalty_shot_socceraction(team_wyid, competition_wyid, season_wyid = ''):
  shot = []
  shots_df = pd.DataFrame(shot)
  games = api.games(competition_wyid, season_wyid)
  for i, m in tq(games.iterrows(), desc = 'Progresso partite processate calcio d\'angolo indiretto', total = len(games)):
    if m['home_team_id'] == team_wyid or m['away_team_id'] == team_wyid:
      events = api.events(m['game_id'])
      if m['home_team_id'] == team_wyid:
        home_team_id = team_wyid
      else:
        home_team_id = m['home_team_id']
      actions = spadl.wyscout.convert_to_actions(events, home_team_id)
      actions = spadl.add_names(actions)
      for j, e in actions.iterrows():
        if e['type_name'] == 'shot_penalty' and e['team_id'] == team_wyid:
          e['shotType'] = 'Penalty'
          shot.append(e)


  shots_df = pd.DataFrame(shot)
  return shots_df

In [ ]:
def get_own_socceraction(team_wyid, competition_wyid, season_wyid = ''):
  shot = []
  shots_df = pd.DataFrame(shot)
  games = api.games(competition_wyid, season_wyid)
  for i, m in tq(games.iterrows(), desc = 'Progresso partite processate calcio d\'angolo indiretto', total = len(games)):
    if m['home_team_id'] == team_wyid or m['away_team_id'] == team_wyid:
      events = api.events(m['game_id'])
      if m['home_team_id'] == team_wyid:
        home_team_id = team_wyid
      else:
        home_team_id = m['home_team_id']
      actions = spadl.wyscout.convert_to_actions(events, home_team_id)
      actions = spadl.add_names(actions)
      for j, e in actions.iterrows():
        if e['type_name'] == 'shot' and e['team_id'] == team_wyid:
          if['result_name'] == 'own_goal':
            e['shotType'] = 'OwnGoal'
            shot.append(e)
            next


  shots_df = pd.DataFrame(shot)
  return shots_df